# Log Viewer (Voila)

In [ ]:
# ===============================
# Imports
# ===============================
from pathlib import Path
from urllib.parse import unquote
import sys
import os
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display, HTML

print("[LOG_VIEWER] Starting cell execution...", flush=True)

# ===============================
# Path handling
# ===============================
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
if current_dir not in sys.path:
    sys.path.append(current_dir)

BASE_DIR = Path(current_dir).resolve()
PARENT_DIR = BASE_DIR.parent

print(f"[LOG_VIEWER] BASE_DIR   = {BASE_DIR}", flush=True)
print(f"[LOG_VIEWER] PARENT_DIR = {PARENT_DIR}", flush=True)

# ===============================
# Log file discovery (SAFE)
# ===============================
def find_log_file() -> Path:
    candidates = [
        PARENT_DIR / "notebook_usage.log",
        BASE_DIR / "notebook_usage.log",
    ]
    for path in candidates:
        if path.exists():
            print(f"[LOG_VIEWER] Found log: {path}", flush=True)
            return path
    raise FileNotFoundError("notebook_usage.log not found")

# ===============================
# Load & preprocess data
# ===============================
def load_log_dataframe(path: Path) -> pd.DataFrame:
    print(f"[LOG_VIEWER] Loading CSV: {path}", flush=True)
    df = pd.read_csv(path)

    required = ["Date", "Time", "User", "App", "File"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    df["File"] = df["File"].astype(str).map(unquote)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Timestamp"] = pd.to_datetime(
        df["Date"].dt.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )

    df = df.dropna(subset=["Timestamp"]).copy()
    df["Day"] = df["Date"].dt.date

    print(f"[LOG_VIEWER] Loaded {len(df)} rows", flush=True)
    return df

print("[LOG_VIEWER] Loading data...", flush=True)
log_path = find_log_file()
df_all = load_log_dataframe(log_path)
print("[LOG_VIEWER] Data ready", flush=True)

# ===============================
# Widgets
# ===============================
print("[LOG_VIEWER] Creating widgets...", flush=True)
min_day = df_all["Day"].min()
max_day = df_all["Day"].max()

start_picker = widgets.DatePicker(description="From", value=min_day)
end_picker   = widgets.DatePicker(description="To", value=max_day)

mode_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df_all["App"].dropna().astype(str).unique().tolist()),
    value="All",
    description="Mode"
)

user_options = sorted(df_all["User"].dropna().astype(str).unique().tolist())
user_select = widgets.SelectMultiple(
    options=user_options,
    value=tuple(user_options),
    rows=min(8, max(4, len(user_options))),
    description="User"
)

script_search = widgets.Text(
    placeholder="Part of script name",
    description="Script"
)

chart_type_dropdown = widgets.Dropdown(
    options=["Bar", "Line + Markers"],
    value="Line + Markers",
    description="Chart"
)
period_dropdown = widgets.Dropdown(
    options=["Daily", "Weekly"],
    value="Daily",
    description="Period"
)
user_sort = widgets.Dropdown(
    options=["Most starts", "User (A-Z)"],
    value="Most starts",
    description="User sort"
)

refresh_btn = widgets.Button(description="Renew", button_style="primary")
out = widgets.Output()

print("[LOG_VIEWER] Widgets created", flush=True)

# ===============================
# Filtering
# ===============================
def filtered_df() -> pd.DataFrame:
    data = df_all.copy()
    if start_picker.value:
        data = data[data["Day"] >= start_picker.value]
    if end_picker.value:
        data = data[data["Day"] <= end_picker.value]
    if mode_dropdown.value != "All":
        data = data[data["App"] == mode_dropdown.value]
    if user_select.value:
        data = data[data["User"].isin(user_select.value)]
    q = script_search.value.strip().lower()
    if q:
        data = data[data["File"].str.lower().str.contains(q, na=False)]
    return data.sort_values("Timestamp")

# ===============================
# Render (non-blocking)
# ===============================
def render(_=None):
    with out:
        out.clear_output(wait=True)
        try:
            data = filtered_df()
            if data.empty:
                display(HTML("<b>No data.</b>"))
                return

            display(HTML(
                f"""<h3>Overview</h3>
                <ul>
                  <li><b>Entries:</b> {len(data)}</li>
                  <li><b>Users:</b> {data['User'].nunique()}</li>
                  <li><b>Scripts:</b> {data['File'].nunique()}</li>
                </ul>"""
            ))

            # User stats
            top_users = data.groupby("User", as_index=False).size().rename(columns={"size": "Starts"})
            if user_sort.value == "Most starts":
                top_users = top_users.sort_values("Starts", ascending=False)
            else:
                top_users = top_users.sort_values("User", key=lambda s: s.astype(str).str.lower())
            display(top_users.head(15))

            # Usage chart

            usage_data = data.copy()
            if period_dropdown.value == "Weekly":
                usage_data["Period"] = usage_data["Timestamp"].dt.to_period("W").apply(lambda p: p.start_time)
                usage_data["PeriodLabel"] = usage_data["Period"].dt.strftime("Week of %Y-%m-%d")
                x_axis_title = "Week"
            else:
                usage_data["Period"] = pd.to_datetime(usage_data["Day"])
                usage_data["PeriodLabel"] = usage_data["Period"].dt.strftime("%Y-%m-%d")
                x_axis_title = "Day"

            period_stats = usage_data.groupby("Period", as_index=False).size().rename(columns={"size": "Starts"})
            period_labels = usage_data.groupby("Period", as_index=False)["PeriodLabel"].first()

            user_dist = usage_data.groupby(["Period", "User"], as_index=False).size().rename(columns={"size": "Starts"})
            user_dist = user_dist.sort_values(["Period", "Starts", "User"], ascending=[True, False, True])
            user_dist["UserStarts"] = user_dist["User"].astype(str) + ": " + user_dist["Starts"].astype(int).astype(str)
            user_dist_text = user_dist.groupby("Period", as_index=False)["UserStarts"].agg("<br>".join)
            user_dist_map = dict(zip(user_dist_text["Period"], user_dist_text["UserStarts"]))

            period_stats = period_stats.merge(period_labels, on="Period", how="left")
            period_stats["UserDistribution"] = period_stats["Period"].map(user_dist_map).fillna("-")

            if chart_type_dropdown.value == "Bar":
                fig_usage = px.bar(
                    period_stats,
                    x="PeriodLabel",
                    y="Starts",
                    category_orders={"PeriodLabel": period_stats["PeriodLabel"].tolist()}
                )
            else:
                fig_usage = px.line(
                    period_stats,
                    x="PeriodLabel",
                    y="Starts",
                    markers=True,
                    category_orders={"PeriodLabel": period_stats["PeriodLabel"].tolist()}
                )
            fig_usage.update_traces(
                customdata=period_stats[["UserDistribution"]].to_numpy(),
                hovertemplate=(
                    "<b>%{x}</b><br>"
                    "Starts: %{y}<br><br>"
                    "User distribution:<br>%{customdata[0]}<extra></extra>"
                )
            )
            fig_usage.update_layout(
                title="Script usage over time",
                xaxis_title=x_axis_title,
                yaxis_title="Number of starts"
            )
            display(HTML(fig_usage.to_html(full_html=False, include_plotlyjs='cdn')))
        except Exception as e:
            display(HTML(f"<b style='color:red;'>Error: {e}</b>"))

# ===============================
# UI Display (NO BLOCKING RENDER)
# ===============================
print("[LOG_VIEWER] Setting up UI...", flush=True)

controls = widgets.VBox([
    widgets.HTML(f"<b>Log file:</b> {log_path.name}"),
    widgets.HBox([start_picker, end_picker, mode_dropdown, chart_type_dropdown, period_dropdown]),
    widgets.HBox([user_select, script_search]),
    user_sort,
    refresh_btn,
])

display(controls, out)

# Wire up observers
for w in [start_picker, end_picker, mode_dropdown, chart_type_dropdown, period_dropdown, user_select, script_search, user_sort]:
    w.observe(render, names="value")
refresh_btn.on_click(render)

print("[LOG_VIEWER] Ready! Click 'Renew' to load data.", flush=True)

[LOG_VIEWER] Starting cell execution...
[LOG_VIEWER] BASE_DIR   = C:\nomad-perotf-jupyter-voila-scripts\log_view
[LOG_VIEWER] PARENT_DIR = C:\nomad-perotf-jupyter-voila-scripts
[LOG_VIEWER] Loading data...
[LOG_VIEWER] Found log: C:\nomad-perotf-jupyter-voila-scripts\notebook_usage.log
[LOG_VIEWER] Loading CSV: C:\nomad-perotf-jupyter-voila-scripts\notebook_usage.log
[LOG_VIEWER] Loaded 1952 rows
[LOG_VIEWER] Data ready
[LOG_VIEWER] Creating widgets...
[LOG_VIEWER] Widgets created
[LOG_VIEWER] Setting up UI...


Output()

[LOG_VIEWER] Ready! Click 'Renew' to load data.
